In [1]:
import numpy as np
import sounddevice as sd
import librosa
import joblib
import os
import tempfile
import warnings
from tkinter import Tk, filedialog

warnings.filterwarnings("ignore")

# Load model and selected features
model = joblib.load("random_forest_model.pkl")
selected_features = joblib.load("selected_features.pkl")  # A list of feature names

# Label mapping
label_mapping = {
    0: "Asphyxia",
    1: "Deaf",
    2: "Hungery",
    3: "Normal",
    4: "Pain"
}

# Suggestions
suggestions = {
    "Asphyxia": "Please check if your baby’s sleeping position is comfortable and their nose and mouth are not covered. If the crying continues or seems unusual, it’s best to consult your doctor.",
    "Deaf": "Your baby might be crying loudly or in an unusual pattern. Try to soothe them in a quiet environment and observe if they respond to sounds. If concerned, consider seeking advice from a specialist.",
    "Hungery": "Your baby may be hungry. Try offering a feeding or their usual meal.",
    "Normal": "Your baby seems to be crying normally. There’s likely nothing to worry about. Keep an eye on them, and if the crying stops soon, all is well.",
    "Pain": "Your baby might be experiencing mild discomfort, like gas or needing a diaper change. Try gently checking and comforting them."
}

# Extract features from audio
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=22050)
    if np.max(np.abs(y)) < 0.01:
        return None  # No valid sound detected
    features_dict = {}
    features_dict["ZeroCrossingRate"] = np.mean(librosa.feature.zero_crossing_rate(y))
    features_dict["SpectralCentroid"] = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    features_dict["SpectralBandwidth"] = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    features_dict["SpectralContrast_1"] = np.mean(contrast[0])
    features_dict["SpectralContrast_3"] = np.mean(contrast[2])
    features_dict["SpectralContrast_6"] = np.mean(contrast[5])
    features_dict["RMSE"] = np.mean(librosa.feature.rms(y=y))
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    for i in range(13):
        features_dict[f"MFCC{i+1}"] = np.mean(mfccs[i])
    # Select only the needed features
    feature_vector = [features_dict[feat] for feat in selected_features if feat in features_dict]
    return feature_vector

# Record audio
def record_audio(duration=10, sample_rate=22050):
    print("Recording...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
    sd.wait()
    temp_path = os.path.join(tempfile.gettempdir(), "temp_audio.wav")
    librosa.output.write_wav(temp_path, audio.flatten(), sample_rate)
    return temp_path

# Predict class
def predict_class(audio_path):
    features = extract_features(audio_path)
    if features is None:
        print("Unable to detect valid sound. Please try again.")
        return
    prediction = model.predict([features])[0]
    class_name = label_mapping[prediction]
    print(f"\n Cause of crying: {class_name}")
    print(f" Recommendation : {suggestions[class_name]}")

# GUI file selection
def upload_audio_file():
    Tk().withdraw()  # Hide the main Tk window
    file_path = filedialog.askopenfilename(filetypes=[("Audio files", "*.wav *.mp3")])
    return file_path

# Main program
def main():
    while True:
        print("\nChoose input method:")
        print("1. Record audio")
        print("2. Upload audio file")
        choice = input("Enter your choice (1/2): ")

        if choice == '1':
            path = record_audio()
            predict_class(path)
        elif choice == '2':
            file_path = upload_audio_file()
            if file_path and os.path.exists(file_path):
                predict_class(file_path)
            else:
                print(" No file selected or file does not exist.")
        else:
            print(" Invalid choice.")

        # Ask if they want to try again
        retry = input("\nDo you want to try again? (y/n): ").strip().lower()
        if retry != 'y':
            print("Goodbye ")
            break

if __name__ == "__main__":
    main()



Choose input method:
1. Record audio
2. Upload audio file
